# GPUs

View the graphics card information

In [1]:
!nvidia-smi

Wed Jul 13 08:05:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   62C    P0    70W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+


|   1  Tesla V100-SXM2...  Off  | 00000000:00:1C.0 Off |                    0 |
| N/A   53C    P0    58W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   2  Tesla V100-SXM2...  Off  | 00000000:00:1D.0 Off |                    0 |
| N/A   53C    P0    81W / 300W |   1570MiB / 16160MiB |     60%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   3  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   52C    P0    84W / 300W |   1582MiB / 16160MiB |     55%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                        

Computing Devices

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

def cpu():  
    return torch.device('cpu')
def gpu(i=0):  
    return torch.device(f'cuda:{i}')
cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

Query the number of available GPUs

In [4]:
def num_gpus():  
    return torch.cuda.device_count()
num_gpus()

2

Define two convenient functions that allow us
to run code even if the requested GPUs do not exist

In [5]:
def try_gpu(i=0):  
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

Query the device where the tensor is located

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

Store a tensor on the GPU

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

Create a random tensor on the second GPU

In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.2081, 0.1938, 0.2107],
        [0.4302, 0.7709, 0.3293]], device='cuda:1')

If we want to compute `X + Y`,
we need to decide where to perform this operation

In [9]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


The data is on the same GPU
(both `Z` and `Y` are),
we can add them up

In [10]:
Y + Z

tensor([[1.2081, 1.1938, 1.2107],
        [1.4302, 1.7709, 1.3293]], device='cuda:1')

In [11]:
Z.cuda(1) is Z

True

Neural Networks and GPUs

In [12]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

/home/d2l-worker/miniconda3/envs/d2l-en-release-1/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [13]:
net(X)

tensor([[0.6670],
        [0.6670]], device='cuda:0', grad_fn=<AddmmBackward0>)

Confirm that the model parameters are stored on the same GPU

In [14]:
net[0].weight.data.device

device(type='cuda', index=0)

In [15]:
@d2l.add_to_class(d2l.Trainer)  
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model